In [1]:
!pip install gradio opencv-python mtcnn tensorflow pillow pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.0 MB/s eta 0:00:00


In [5]:
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from mtcnn.mtcnn import MTCNN
from PIL import Image
import gradio as gr
from datetime import datetime
import os
from google.colab import drive

drive.mount('/content/drive')

# -------- LOAD MODEL --------
MODEL_PATH = "/content/drive/MyDrive/DermalScan_project/DermalScan_Model.h5"
model = load_model(MODEL_PATH)

classes = ["wrinkles", "dark_spots", "puffy_eyes", "clear_skin"]
detector = MTCNN()

# -------- MAIN FUNCTION --------
def dermalscan(image):
    img = np.array(image)
    detections = detector.detect_faces(img)

    results = []
    annotated = img.copy()

    if len(detections) == 0:
        return image, None, None

    for face in detections:
        x, y, w, h = face["box"]
        x, y = abs(x), abs(y)

        face_img = img[y:y+h, x:x+w]
        face_img = cv2.resize(face_img, (224, 224))
        face_img = face_img / 255.0
        face_img = face_img.reshape(1, 224, 224, 3)

        pred = model.predict(face_img, verbose=0)[0]
        label = classes[np.argmax(pred)]
        confidence = round(float(np.max(pred) * 100), 2)

        cv2.rectangle(annotated, (x,y), (x+w,y+h), (0,255,0), 2)
        cv2.putText(
            annotated,
            f"{label} ({confidence}%)",
            (x, y-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0,255,0),
            2
        )

        results.append({
            "Prediction": label,
            "Confidence (%)": confidence,
            "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })

    df = pd.DataFrame(results)

    csv_path = "DermalScan_Report.csv"
    df.to_csv(csv_path, index=False)

    return Image.fromarray(annotated), df, csv_path

# -------- GRADIO UI --------
with gr.Blocks() as demo:
    gr.Markdown("## 🧬 DermalScan – Skin Aging Detection")
    gr.Markdown("Upload an image (single or group photo). Faces will be detected and classified.")

    with gr.Row():
        inp = gr.Image(type="pil", label="Upload Face Image")
        out_img = gr.Image(label="Detected Faces")

    out_table = gr.Dataframe(label="Prediction Results")
    out_file = gr.File(label="Download CSV Report")

    btn = gr.Button("Analyze Image")

    btn.click(
        fn=dermalscan,
        inputs=inp,
        outputs=[out_img, out_table, out_file]
    )

demo.launch(share=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://684af5485c5b414978.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
